In [19]:
import numpy as np

def is_close_boxes(box1:dict = None, box2:dict = None, min_distance:int = 0, log_view:bool = False):
    """
    두 박스의 거리가 좁은지 확인합니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
        min_distance: 최소 거리
    
    Returns:
        True if 두 박스의 거리가 좁으면, False otherwise
    """
    
    box1_center_x = (box1["x"] + box1["w"]) / 2
    box1_center_y = (box1["y"] + box1["h"]) / 2

    
    box2_center_x = (box2["x"] + box2["w"]) / 2
    box2_center_y = (box2["y"] + box2["h"]) / 2
    
    # 두 박스의 중심점 간의 거리를 계산합니다.
    distance = np.sqrt(
        ((box1_center_x - box2_center_x)**2) +
        ((box1_center_y - box2_center_y)**2)
    )

    if log_view:
        print(distance, distance <= min_distance)

    return distance <= min_distance

In [20]:
def merge_boxes(box1, box2):
    """
    두 박스를 하나의 박스로 합칩니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
    
    Returns:
        합쳐진 박스
    """
    
    return {
        "x": min(box1["x"], box2["x"]), # X
        "y": min(box1["y"], box2["y"]), # Y
        "w": max(box1["x"] + box1["w"], box2["x"] + box2["w"]) - min(box1["x"], box2["x"]), # W
        "h": max(box1["y"] + box1["h"], box2["y"] + box2["h"]) - min(box1["y"], box2["y"]), # H
    }

In [21]:
def is_intersecting(box1: dict, box2: dict):
  """
  두 박스가 겹쳐지는지 확인합니다.

  Args:
    box1: 첫 번째 박스
    box2: 두 번째 박스

  Returns:
    True if 두 박스가 겹치면, False otherwise
  """

  return (
    box2["x"] < box2["x"] + box2["w"] and
    box2["x"] < box1["x"] + box1["w"] and
    box2["y"] < box2["y"] + box2["h"] and
    box2["y"] < box1["y"] + box1["h"]
  )

In [22]:
def group_boxes(boxes, min_distance):
  """
  겹쳐지거나 가까운 박스들을 그룹으로 묶습니다.

  Args:
    boxes: 박스 리스트
    min_distance: 박스간의 최소 거리

  Returns:
    그룹화된 박스 리스트
  """

  merged_boxes = []
  for i in range(len(boxes)):
    merged_box = boxes[i]
      
    for j in range(i+1, len(boxes)):
      if is_intersecting(boxes[i], boxes[j]): # 두 박스가 겹쳐지는지 확인합니다.
        merged_box = merge_boxes(boxes[i], boxes[j]) # 겹쳐지는 박스를 하나로 합칩니다.
        merged_boxes.append(merged_box)

  return merged_boxes

In [23]:
import cv2

image = cv2.imread("../dataset_dir/13.한국어글자체/01.손글씨/01_handwriting_sentence_images/1_sentence/00000002.png")
image2 = image.copy()

# 컨투어 찾기 전 이미지 전처리
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
gray = cv2.bitwise_not(gray) # 객체보다 배경이 밝은 경우 이미지 반전

# threshold 등 다양한 전처리 시도 -> 객체와 배경을 가장 잘 분리하는 전처리 사용
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

# 컨투어 찾기
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [24]:
processed_boxes = []
original_boxes = []

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    
    original_boxes.append({
        "x": x, 
        "y": y, 
        "w": w, 
        "h": h
    })
    processed_boxes.append({
        "x": x, 
        "y": y, 
        "w": (x + w), 
        "h": (y + h)
    })

original_boxes = sorted(original_boxes, key=lambda country: country["x"])
processed_boxes = sorted(processed_boxes, key=lambda country: country["x"])

In [25]:
grouped_boxes = group_boxes(processed_boxes, 10)

In [26]:
# 거리가 좁은 박스를 하나의 박스로 합칩니다.
# merged_boxes = []
# for i in range(len(processed_boxes)):
#     if is_close_boxes(processed_boxes[i], processed_boxes[i+1], 50, True):
#         merged_box = merge_boxes(processed_boxes[i], processed_boxes[i+1])
#         merged_boxes.append(merged_box)

In [27]:
# 합쳐진 박스를 표기합니다.
for box in grouped_boxes:
    cv2.rectangle(image, (box["x"], box["y"]), (box["w"], box["h"]), (0, 255, 0), 2)

# 결과 이미지 출력
cv2.imwrite(f'./image/asdf.jpg', image)

True